# SPR 2026 - Word2Vec NILC (Pré-treinado)

**NILC Word Embeddings (NILC-USP)**

Embeddings pré-treinados em corpus massivo de português brasileiro.
Se não disponível, treina Word2Vec do zero com mais epochs.

- ✅ NILC ou fallback Word2Vec local
- ✅ Tempo esperado: ~2 min

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. (Opcional) Add Data → NILC Word Embeddings
3. **IMPORTANTE:** Execute "Run All" após commit
---

In [ ]:
# =============================================================================
# SPR 2026 - WORD2VEC NILC PRE-TREINADO (CÓDIGO CONSOLIDADO)
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors, Word2Vec
import os
import re
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Word2Vec NILC (Pré-treinado)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
EMBEDDING_DIM = 100
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
NILC_PATH = '/kaggle/input/nilc-word-embeddings/cbow_s100.txt'
np.random.seed(SEED)

# ==== CARREGAR DADOS ====
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# ==== PREPROCESSAMENTO ====
print("\n[2/5] Preprocessando textos...")
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
    return text.split()

train['tokens'] = train['report'].apply(preprocess)
test['tokens'] = test['report'].apply(preprocess)

# ==== CARREGAR/TREINAR EMBEDDINGS ====
print("\n[3/5] Carregando embeddings...")
USE_PRETRAINED = False

try:
    if os.path.exists(NILC_PATH):
        print("    Carregando NILC embeddings pré-treinados...")
        w2v = KeyedVectors.load_word2vec_format(NILC_PATH)
        USE_PRETRAINED = True
        print(f"    ✓ NILC vocabulário: {len(w2v)} palavras")
except Exception as e:
    print(f"    NILC não disponível: {e}")

if not USE_PRETRAINED:
    print("    Treinando Word2Vec do zero (fallback)...")
    all_texts = train['tokens'].tolist() + test['tokens'].tolist()
    w2v_model = Word2Vec(
        sentences=all_texts,
        vector_size=EMBEDDING_DIM,
        window=5,
        min_count=1,
        workers=4,
        epochs=20,
        seed=SEED
    )
    w2v = w2v_model.wv
    print(f"    ✓ Vocabulário local: {len(w2v)} palavras")

# ==== GERAR EMBEDDINGS ====
print("\n[4/5] Gerando embeddings (Mean Pooling)...")
def text_to_embedding(tokens, model, dim):
    vectors = [model[w] for w in tokens if w in model]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

X_train = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
X_test = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in test['tokens']])
y_train = train['target'].values
print(f"    X_train shape: {X_train.shape}")

# ==== LOGISTIC REGRESSION ====
print("\n[5/5] Treinando Logistic Regression...")
model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("    ✓ Modelo treinado!")

# ==== SUBMISSÃO ====
predictions = model.predict(X_test)
submission = pd.DataFrame({'ID': test['ID'], 'target': predictions})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())